In [206]:
import requests, urllib.request

import re

from lxml import html

import pandas as pd

from lxml import etree
 

SCRIPT_DIR = "C:\\DEV\\output\\"

 
def get_data():
    url="http://www.meteociel.fr/temps-reel/obs_villes.php"

    response = requests.get(url)

    page = html.fromstring(response.text)

    attribute1 = page.xpath('//select[@id="paysdept"]/option')

    out2 = [e.get("value").replace('\n','') for e in attribute1]

    regions = pd.DataFrame({'name'  :  out2})

    regions = regions[~(regions.name.str.startswith('dept')) & ~(regions.name=='none') & ~(regions.name=='') ]



    #Loop through each regions

    url = "http://www.meteociel.fr/cartes_obs/stationselect_helper.php?deptpays="

    dataTbl = pd.DataFrame()

    for i in range(len(regions)) :

        region_name = regions.name.iloc[i]

        print("DEBUG-location: *" + str(i) + "* for '" + region_name + "'" )

        response = requests.get( url + region_name )

        page = html.fromstring(response.text)

        el = page.xpath('//body')

        json_str = el[0].text_content()

        data = [[e.replace(' ','').replace('\n','') for e in row.split('|')] for row in json_str.split('\n')]

        data = pd.DataFrame(data)

        data = {'region'    :  region_name,

              'location'  :  data[1].map(lambda x: format_name(x).replace( ")" , "" ).replace( "(" , "_" ).capitalize() if x else "NaN"),

              'code_2'    :  data[0]}

        data = pd.DataFrame(data)

        data = data.dropna()

        dataTbl = dataTbl.append(data)

        dataTbl = dataTbl[~(dataTbl.location == 'NaN') ]
        
            
    dataTbl.to_csv(SCRIPT_DIR + "locations_list.csv", sep=';',index=False)
    
    
    
    
def format_name(str):

    dic ={
            u'\xe1': 'a', u'\xe2': 'a', u'\xe3': 'a', u'\xe4': 'a', u'\xe5': 'a', u'\xe6': 'a',
        
            u'\xe7': 'c',
        
            u'\xe8': 'e', u'\xe9': 'e', u'\xea': 'e', u'\xeb': 'e',
        
            u'\xf1': 'n',
        
            u'\xf2': 'o', u'\xf3': 'o', u'\xf4': 'o', u'\xf6': 'o',
        
            u'\xed': 'i', u'\xee': 'i',
        
            u'\xfa': 'u', u'\xfb': 'u', u'\xfc': 'u',
        
            u'\xf0': 'd',
        
            u'\xfb': 'th',
        
            u'\xfd': 'y',
        
            '\\': '-', '/': '-','#': '-',
        
            "'": '', '?': '', '\x92': '', '=': '', '*': '', ',': '',
        }
    
    for d in dic:
        str = str.lower().replace(d,dic[d])

    return str




def main():

    get_data()
    


if __name__ == '__main__':
    main()

DEBUG-location: *0* for 'france'
DEBUG-location: *1* for 'germany'
DEBUG-location: *2* for 'austria'
DEBUG-location: *3* for 'belgium'
DEBUG-location: *4* for 'croatia'
DEBUG-location: *5* for 'denmark'
DEBUG-location: *6* for 'spain'
DEBUG-location: *7* for 'finland'
DEBUG-location: *8* for 'greenland'
DEBUG-location: *9* for 'greece'
DEBUG-location: *10* for 'hungary'
DEBUG-location: *11* for 'ireland'
DEBUG-location: *12* for 'iceland'
DEBUG-location: *13* for 'italy'
DEBUG-location: *14* for 'luxembourg'
DEBUG-location: *15* for 'norway'
DEBUG-location: *16* for 'netherlands'
DEBUG-location: *17* for 'poland'
DEBUG-location: *18* for 'portugal'
DEBUG-location: *19* for 'romania'
DEBUG-location: *20* for 'united_kingdom'
DEBUG-location: *21* for 'ukraine'
DEBUG-location: *22* for 'slovakia'
DEBUG-location: *23* for 'slovenia'
DEBUG-location: *24* for 'sweden'
DEBUG-location: *25* for 'switzerland'
DEBUG-location: *26* for 'antarctica'
DEBUG-location: *27* for 'australia'
DEBUG-locat

In [2]:
a = "à"
# print(a.lower())
print(a.encode('unicode_escape'))

# re.compile(r'\\')



b = u'\xe0'

# print(u"F\xfcllungsgrad der Speicherseen")

# c = "ð"

print(a==b)

# print(b)

b'\\xe0'
True


In [185]:
import requests, urllib.request
from lxml import etree, html
import csv
import pandas as pd
from datetime import datetime,timedelta,time
import re
import pytz



__author__ = 'aituarov'



DB_LOADER_DIR = "C:\\DEV\\output\\"

#******************************     Giving input data as csv file and start,end dates    *************************************#


csv_path = DB_LOADER_DIR + "locations_list.csv"
csv_reader = csv.reader(open(csv_path, "r"), delimiter =";")
header = next(csv_reader)


end_date = datetime.today().date() + timedelta(days=-1)
start_date = end_date + timedelta(days=-3)


needed_values = {'temperature': 'C', 'humidite': 'percent', 'vent': 'km/h', 'precip': 'mm'}
post_fix = ' – meteocielEU weather data act'




#******************************     Reading csv file line by line    *************************************#
def get_data():
    
    data_table = pd.DataFrame()
    
    is_hour_utc = False

    for line in csv_reader:
        
        station_name_csv = line[1]
        station_code_csv = line[2]

        request_date = start_date

#         Creating url just for check
        url = "http://www.meteociel.fr/temps-reel/obs_villes.php?code2="+str(station_code_csv)+"&jour2="+str(request_date.day)+"&mois2="+str(request_date.month-1)+"&annee2="+str(request_date.year)
        response = requests.get(url) 
        page = html.fromstring(response.text)
        
#         try:
#             selected_station_site = format_name(page.xpath('//option[@value="' + station_code_csv + '"]')[0].text_content()).replace(")" , "").replace(" ", '').replace("(" , "_")

# #         #******************************     Checking the station_name in file with written in the title    **********************#

#         if ((station_name_csv.lower()) != selected_station_site):
#             print("Please check station names!!!")
                
                
            
        
        while request_date <= end_date:
        # Creating url every time, when requestDate is change
            url = "http://www.meteociel.fr/temps-reel/obs_villes.php?code2="+str(station_code_csv)+"&jour2="+str(request_date.day)+"&mois2="+str(request_date.month - 1)+"&annee2="+str(request_date.year)
            print("Retrieving data from: " + url)
            response = requests.get(url) 
            page = html.fromstring(response.text)

            element_table = page.xpath('//td/b[contains(.,"Heure")]/../../..')[0]
            
            if request_date == start_date:
                header = [(format_name(td.text_content())).replace('.','').split()[0] for td in element_table.xpath('./tr[1]/td')]
                print(header)
                if 'vent' in header:
                    header.insert(header.index('vent'), 'vent_direction')
                
                if header[0] == "heureutc":
                    is_hour_utc = True
                else:
                    is_hour_utc = False

                header[0] = 'hour'


            data = [[td.text_content().strip().replace(' ','').replace("&nbsp", '').replace('\n','').split('(')[0] for td in tr] for tr in element_table.xpath('./tr[position()>1]')]
            data = pd.DataFrame(data, columns=header)

            for column in header:
                if not (column in needed_values.keys() or column == 'hour'):
                    data = data.drop(column, axis=1)

            data = pd.melt(data, id_vars=['hour'], value_vars=needed_values)
            
        
            def get_start_date(contract):
                if is_hour_utc:
                    try:
                        ct = datetime.combine(request_date, time(int(re.findall('\d+', contract)[0])))
                        tz_utc = pytz.timezone('UTC')
                        ct_utc = tz_utc.localize(ct)

                        tz_local = pytz.timezone(line[3])

                        local_start_date = ct_utc.astimezone(tz_local)
                        local_start_date_str = local_start_date.date().strftime('%Y-%m-%d ') + str(local_start_date.time())

                    except Exception as e:
                        print('ERROR-converting-hourUTC-to-hourLocale : ' + str(e))

                else:
                    local_start_date_str = str(request_date) + ' ' + str(re.findall('\d+', contract)[0])+ ':00:00'

                return local_start_date_str
            
            
            data = {
                    'name': data['variable'].map(lambda x: station_name_csv + ' ' + x + '_' + needed_values[x] + post_fix),
                    'effective_date': '',
                    'start_date': data['hour'].map(lambda x: get_start_date(x)),
                    'end_date': '',
                    'value' :  data['value'].map(lambda x: re.findall(r'[\d\.\d]+', x)[0] if re.findall(r'[\d\.\d]+', x)!=[] else 0 ),
                    'table': 'spot'
                    }
            
            data = pd.DataFrame(data)
            data_table = data_table.append(data)

            request_date += timedelta(days=1) #Changing date to the next, until I will get end_date
        
        print()
            
    data_table.to_csv(DB_LOADER_DIR + "station_values", sep=';',index=False)
    print("End.")






def format_name(str):

    dic ={
            u'\xe1': 'a', u'\xe2': 'a', u'\xe3': 'a', u'\xe4': 'a', u'\xe5': 'a', u'\xe6': 'a',
        
            u'\xe7': 'c',
        
            u'\xe8': 'e', u'\xe9': 'e', u'\xea': 'e', u'\xeb': 'e',
        
            u'\xf1': 'n',
        
            u'\xf2': 'o', u'\xf3': 'o', u'\xf4': 'o', u'\xf6': 'o',
        
            u'\xed': 'i', u'\xee': 'i',
        
            u'\xfa': 'u', u'\xfb': 'u', u'\xfc': 'u',
        
            u'\xf0': 'd',
        
            u'\xfb': 'th',
        
            u'\xfd': 'y',
        
            '\\': '-', '/': '-','#': '-',
        
            "'": '', '?': '', '\x92': '', '=': '', '*': '', ',': '',
        }
    
    for d in dic:
        str = str.lower().replace(d,dic[d])

    return str




def main():    

    get_data()
    



if __name__ == '__main__':
    main()

    
    
#CHUNK SIZE

Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=7005&jour2=16&mois2=5&annee2=2020
['heurelocale', 'neb', 'temps', 'visi', 'temperature', 'humidite', 'humidex', 'windchill', 'vent', 'pression', 'precip']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=7005&jour2=17&mois2=5&annee2=2020
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=7005&jour2=18&mois2=5&annee2=2020
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=7005&jour2=19&mois2=5&annee2=2020

Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=78005002&jour2=16&mois2=5&annee2=2020
['heurelocale', 'neb', 'temps', 'visi', 'temperature', 'humidite', 'humidex', 'windchill', 'vent', 'pression', 'precip']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=78005002&jour2=17&mois2=5&annee2=2020
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2

KeyboardInterrupt: 

In [31]:
from lxml import etree
import csv
import pandas as pd
from lxml import html
from datetime import datetime,timedelta
import re



__author__ = 'aituarov'



DB_LOADER_DIR = "C:\\DEV\\output\\"

#******************************     Giving input data as csv file and start,end dates    *************************************#


csv_path = DB_LOADER_DIR + "locations_list.csv"
csv_reader = csv.reader(open(csv_path, "r"), delimiter =";")
header = next(csv_reader)

for rows in csv_reader:
    station_name = rows[1]
    code = rows[2]
    print(rows[3])

IndexError: list index out of range

In [37]:
from datetime import datetime, timedelta

import pytz

ct = datetime.now() + timedelta(hours=-6)

tz_utc = pytz.timezone('UTC')

ct_utc = tz_utc.localize(ct)

tz_local = pytz.timezone("Asia/Oral")

ct_local = ct_utc.astimezone(tz_local)

2020-06-19 12:31:07.534465
UTC
2020-06-19 12:31:07.534465+00:00
2020-06-19 17:31:07.534465+05:00


In [164]:
# import datetime

from datetime import date,datetime,timedelta,time

contract = '23h'

ct = datetime.combine(end_date, time(int(re.findall('\d+', contract)[0])))

print(ct)

# print(end_date)

# print(type(date))

tz_utc = pytz.timezone('UTC')

ct_utc = tz_utc.localize(ct)

tz_local = pytz.timezone("Asia/Oral")

ct_local = ct_utc.astimezone(tz_local)

date = ct_local.date()

time = ct_local.time()

# print(date)

# print(time)

# str1 = date.strftime('%Y-%m-%d')



# print(str1)



# local_start_date = date.

# local_start_date = ct_local.date()

# ct_local.time()



# print(datetime.timezone(ct_local))
# ct_local = datetime.strftime(ct_local, '%Y-%m-%d %H:%M:%S')

# ct_local.strptime(ct'%Y')

# ct_local('%Y')

2020-06-18 23:00:00


In [16]:
from datetime import datetime,timedelta,date

start_date = datetime.today().date()

date_range = 7

for single_date in (start_date + timedelta(n) for n in range(date_range)):
    print(single_date)


print(delta)

# print(end_date)
# print(start_date)

2020-06-25
2020-06-26
2020-06-27
2020-06-28
2020-06-29
2020-06-30
2020-07-01
-4 days, 0:00:00


In [138]:
import pandas as pd

from datetime import datetime,timedelta,date

import os,time

import requests

from shutil import move

 

__author__ = 'berclc'

 

 

DB_LOADER_DIR = "R:\\Trading\\Prop\\stdloader\\pwr_raw\\"

DATASET_DIR = "R:\\Trading\\Prop\\scrapes\\datafiles\\"

DATASET_DIR = DATASET_DIR + "ren_PT\\"

SCRAPE_INTERVAL = 30

CHUNK_SIZE=7

 

today_dt = datetime.today()

 

start_date = today_dt-timedelta(SCRAPE_INTERVAL)
# 20-05-2020

end_date = today_dt-timedelta(1)
# 19-06-2020

 

#start_date = datetime.strptime( "2008-01-01", "%Y-%m-%d" )

#end_date = datetime.strptime( "2019-02-24", "%Y-%m-%d" )

 

 

def retrieve_data():

    dts = start_date
# 20-05-2020

    dte = min(dts+timedelta(CHUNK_SIZE),end_date)
# min(27-05-2020, 19-06-2020) = 27-05-2020
    

    while dts <= end_date:#20-05-2020 <= 19-06-2020

 

        dte = min(dts+timedelta(CHUNK_SIZE),end_date)# min(27-05-2020, 19-06-2020) = 27-05-2020

        print("Downloading from '"+dts.strftime('%Y-%m-%d') + "' -> '" + dte.strftime('%Y-%m-%d') + "'" )

        url = "http://www.mercado.ren.pt/UserPagesFase2/Dados_download.aspx?Dia1=" + dts.strftime('%Y-%m-%d') + "&Dia2=" + dte.strftime('%Y-%m-%d') + "&Nome=ProdVerificadaElectr"

        #http://www.mercado.ren.pt/UserPagesFase2/Dados_download.aspx?Dia1=2019-12-01&Dia2=2019-12-11&Nome=ProdVerificadaElectr

 

        r = requests.get(url, verify=False)

        time.sleep(4)

        f = open(DATASET_DIR + "gen_archive_" + dts.strftime('%Y%m%d') + "_" + dte.strftime('%Y%m%d') + ".xls", "w")

        f.write(r.text)

        f.close()

        dts = dte+timedelta(1)# 28-05-2020

 

 

def write_stdldr(data,f_id):

    data['effective_date'] = ''

    data['start_date'] = data['value_date'].map(lambda x: datetime.strftime(x,'%Y-%m-%d %H:%M:%S'))

    data['end_date'] = ''

    data['table'] = 'spot'

    data = data[["name","effective_date","start_date","end_date","value","table"]]

    outfile = DB_LOADER_DIR + "py_renPT_generation_("+str(f_id)+")_"+ today_dt.strftime('%Y_%m_%d_%H%M%S')+".csv"

    data.to_csv(outfile, sep=';',index=False)

 

 

def read_data():

    f_id = 0

    cnt = 0

    flist = [f for f in os.listdir(DATASET_DIR) if (f.startswith('gen_archive_') & f.endswith('.xls'))]

    for fname in flist:

        try:

            cnt = cnt + 1

            if cnt > 30//CHUNK_SIZE+1:

                cnt = 0

                f_id = f_id+1

                if 'total' in locals():

                    write_stdldr(total,f_id)

                    del total

                    print('flushed')

 

            print(fname)

            with open(DATASET_DIR + fname, 'r') as f:

                  content = f.read()

            print('start to process the data')

            data = pd.read_html(content, header=0,thousands="'",decimal=",")[0]

            #converters = {column_name: str for column_name in data.dtypes.index}

            #data = pd.read_html(content, header=0, converters=converters)[0]

 

            data = data.dropna(subset=['DATE'])

 

            #day-light

            data['count'] = data.groupby('DATE', as_index=False)['DATE'].transform(lambda s: s.count())

            data['HOUR']=data['HOUR'].astype(int)

            data['HOUR_']=float('NaN')

            idx=(data['count']==25) & (data['HOUR']>3)

            data.loc[idx,'HOUR_']=data['HOUR'][idx]-1

            idx=(data['count']==23) & (data['HOUR']>2)

            data.loc[idx,'HOUR_']=data['HOUR'][idx]+1

            idx=(data['HOUR_'].isnull())

            data.loc[idx,'HOUR_']=data['HOUR'][idx]

            #data[data['HOUR_']!=data['HOUR']]

 

            data['value_date']=[ datetime.strptime(date, "%d-%m-%Y%H") for date in  data['DATE']+(data['HOUR_'].astype(int)-1).astype(str) ]

            data = data.drop(['DATE', 'HOUR','HOUR_','count'], axis=1)

            data = pd.melt(data, id_vars=['value_date'],var_name='name',value_name='value')

            data = data.dropna()

            data = data[data['value'].apply(lambda x: isinstance(x, (int,float)))]

            data = data.groupby(['name','value_date'],as_index=False)['value'].mean()

            data['name'] = data['name'].str.lower() + ' - renPT pwr generation act'

            if 'total' in locals():

                print("merging " + fname)

                total = pd.concat([total, data])

            else:

                total = data

            move(DATASET_DIR + fname, DATASET_DIR + 'processed\\' + fname)

 

        except Exception as e:

            print('parsing error : ' + str(e))

 

    if 'total' in locals():

        f_id = f_id+1

        write_stdldr(total,f_id)

 

 

def main():

    print("Retrieve...")

    retrieve_data()

    read_data()

    print("Scrape successfully ended.")

 

if __name__ == '__main__':

    main()


TypeError: descriptor 'time' requires a 'datetime.datetime' object but received a 'int'

In [210]:
import requests, urllib.request
from lxml import etree, html
import csv
import pandas as pd
from datetime import datetime,timedelta,time
import re
import pytz



__author__ = 'aituarov'



DB_LOADER_DIR = "C:\\DEV\\output\\"

#******************************     Giving input data as csv file and start,end dates    *************************************#


csv_path = DB_LOADER_DIR + "locations_list.csv"
open_file = open(csv_path, "r")
csv_reader = csv.reader(open_file, delimiter =";")
header_csv = next(csv_reader)


end_date = datetime.today().date() + timedelta(days=-1)
start_date = end_date + timedelta(days=-1)


needed_values = {'temperature': 'C', 'humidite': 'percent', 'vent': 'km/h', 'precip': 'mm'}
post_fix = ' – meteocielEU weather data act'

CHUNK_SIZE=1



#******************************     Reading csv file line by line    *************************************#
def get_data():
    request_date = start_date
    dte_start = start_date
    
    is_hour_utc = False
    
    data_table = pd.DataFrame()
    f_id = 1
    cnt_days=0
    
    while request_date <= end_date:
        print(request_date)

        for line in csv_reader:
            print(line)

            station_name_csv = line[1]
            station_code_csv = line[2]

    # Creating url every time, when requestDate is change
            url = "http://www.meteociel.fr/temps-reel/obs_villes.php?code2="+str(station_code_csv)+"&jour2="+str(request_date.day)+"&mois2="+str(request_date.month - 1)+"&annee2="+str(request_date.year)
            print("Retrieving data from: " + url)
            response = requests.get(url) 
            page = html.fromstring(response.text)

            element_table = page.xpath('//td/b[contains(.,"Heure")]/../../..')[0]

            header = [(format_name(td.text_content())).replace('.','').split()[0] for td in element_table.xpath('./tr[1]/td')]

            if 'vent' in header:
                header.insert(header.index('vent'), 'vent_direction')

            if header[0] == "heureutc":
                is_hour_utc = True
            else:
                is_hour_utc = False

            header[0] = 'hour'


            data = [[td.text_content().strip().replace(' ','').replace("&nbsp", '').replace('\n','').split('(')[0] for td in tr] for tr in element_table.xpath('./tr[position()>1]')]
            data = pd.DataFrame(data, columns=header)

            for column in header:
                if not (column in needed_values.keys() or column == 'hour'):
                    data = data.drop(column, axis=1)

            data = pd.melt(data, id_vars=['hour'], value_vars=needed_values)


            def get_start_date(contract):
                if is_hour_utc:
                    try:
                        ct = datetime.combine(request_date, time(int(re.findall('\d+', contract)[0])))
                        tz_utc = pytz.timezone('UTC')
                        ct_utc = tz_utc.localize(ct)

                        tz_local = pytz.timezone(line[3])

                        local_start_date = ct_utc.astimezone(tz_local)
                        local_start_date_str = local_start_date.date().strftime('%Y-%m-%d ') + str(local_start_date.time())

                    except Exception as e:
                        print('ERROR-converting-hourUTC-to-hourLocale : ' + str(e))

                else:
                    local_start_date_str = str(request_date) + ' ' + str(re.findall('\d+', contract)[0])+ ':00:00'

                return local_start_date_str


            data = {
                    'name': data['variable'].map(lambda x: station_name_csv + ' ' + x + '_' + needed_values[x] + post_fix),
                    'effective_date': '',
                    'start_date': data['hour'].map(lambda x: get_start_date(x)),
                    'end_date': '',
                    'value' :  data['value'].map(lambda x: re.findall(r'[\d\.\d]+', x)[0] if re.findall(r'[\d\.\d]+', x)!=[] else 0 ),
                    'table': 'spot'
                    }

            data = pd.DataFrame(data)
            data_table = data_table.append(data)

        request_date += timedelta(days=1) #Changing date to the next, until I will get end_date
        open_file.seek(0)
        header_csv = next(csv_reader)
        cnt_days+=1
    
        if cnt_days == CHUNK_SIZE:
            data_table.to_csv(DB_LOADER_DIR + "station_values_( " + str(f_id) + " )" , sep=';',index=False)
            data_table = data_table.iloc[0:0]
            dte_start = request_date
            cnt_days=0
            f_id+=1
        

    
    print("End.")






def format_name(str):

    dic ={
            u'\xe1': 'a', u'\xe2': 'a', u'\xe3': 'a', u'\xe4': 'a', u'\xe5': 'a', u'\xe6': 'a',
        
            u'\xe7': 'c',
        
            u'\xe8': 'e', u'\xe9': 'e', u'\xea': 'e', u'\xeb': 'e',
        
            u'\xf1': 'n',
        
            u'\xf2': 'o', u'\xf3': 'o', u'\xf4': 'o', u'\xf6': 'o',
        
            u'\xed': 'i', u'\xee': 'i',
        
            u'\xfa': 'u', u'\xfb': 'u', u'\xfc': 'u',
        
            u'\xf0': 'd',
        
            u'\xfb': 'th',
        
            u'\xfd': 'y',
        
            '\\': '-', '/': '-','#': '-',
        
            "'": '', '?': '', '\x92': '', '=': '', '*': '', ',': '',
        }
    
    for d in dic:
        str = str.lower().replace(d,dic[d])

    return str




def main():    

    get_data()
    



if __name__ == '__main__':
    main()


2020-06-18
['france', 'Abbeville_80', '7005', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=7005&jour2=18&mois2=5&annee2=2020
['france', 'Acheres_78', '78005002', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=78005002&jour2=18&mois2=5&annee2=2020
['france', 'Adast_65', '65001001', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=65001001&jour2=18&mois2=5&annee2=2020
['france', 'Agde_34', '20', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=20&jour2=18&mois2=5&annee2=2020
['france', 'Agen_47', '7524', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=7524&jour2=18&mois2=5&annee2=2020
['france', 'Aicirits_64', '64010002', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=64010002&jour2=18&mois2=5&annee2=2020
['france', 'Ai

['france', 'Angers-marce_49', '7228', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=7228&jour2=18&mois2=5&annee2=2020
['france', 'Angersville_49', '49007011', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=49007011&jour2=18&mois2=5&annee2=2020
['france', 'Anglarsst-felix_12', '148', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=148&jour2=18&mois2=5&annee2=2020
['france', 'Angliers_17', '25', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=25&jour2=18&mois2=5&annee2=2020
['france', 'Angouleme-briechampniers_16', '7420', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=7420&jour2=18&mois2=5&annee2=2020
['france', 'Angrie_49', '49008001', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=49008001&jour2=18&mois2=5&annee2=2

['france', 'Aumontaubrac_48', '48009001', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=48009001&jour2=18&mois2=5&annee2=2020
['france', 'Aups_83', '83007004', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=83007004&jour2=18&mois2=5&annee2=2020
['france', 'Auray_56', '56007001', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=56007001&jour2=18&mois2=5&annee2=2020
['france', 'Aurelle-verlac_12', '12014001', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=12014001&jour2=18&mois2=5&annee2=2020
['france', 'Aurillac_15', '7549', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=7549&jour2=18&mois2=5&annee2=2020
['france', 'Autun_71', '71014004', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=71014004&jour2=18&mois2=5&annee2

['france', 'Beaumont_37', '37023002', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=37023002&jour2=18&mois2=5&annee2=2020
['france', 'Beaumontdulac_87', '87009003', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=87009003&jour2=18&mois2=5&annee2=2020
['france', 'Beaumontduventoux_montserein_84', '84015002', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=84015002&jour2=18&mois2=5&annee2=2020
['france', 'Beaurepaire_38', '38034001', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=38034001&jour2=18&mois2=5&annee2=2020
['france', 'Beaurieux_02', '2058001', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=2058001&jour2=18&mois2=5&annee2=2020
['france', 'Beauvais_60', '7055', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=7

['france', 'Bezierscapdagde_34', '7638', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=7638&jour2=18&mois2=5&annee2=2020
['france', 'Beziers-courtade_34', '34032002', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=34032002&jour2=18&mois2=5&annee2=2020
['france', 'Biache-saint-vaast_62', '140', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=140&jour2=18&mois2=5&annee2=2020
['france', 'Biarritz_64', '7602', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=7602&jour2=18&mois2=5&annee2=2020
['france', 'Bidache_64', '64123001', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=64123001&jour2=18&mois2=5&annee2=2020
['france', 'Bignan_56', '56017003', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=56017003&jour2=18&mois2=5&an

...

['france', 'Courdimanche_91', '91184001', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=91184001&jour2=18&mois2=5&annee2=2020
['france', 'Courouvre_55', '55129001', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=55129001&jour2=18&mois2=5&annee2=2020
['france', 'Courpiere_63', '63125002', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=63125002&jour2=18&mois2=5&annee2=2020
['france', 'Coursegoules_06', '6050002', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=6050002&jour2=18&mois2=5&annee2=2020
['france', 'Cran-gevrier_74', '74093001', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=74093001&jour2=18&mois2=5&annee2=2020
['france', 'Creil_60', '7057', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=7057&jour2=18&mois2

['france', 'Dourdan_91', '91200002', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=91200002&jour2=18&mois2=5&annee2=2020
['france', 'Dourgne_81', '81081002', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=81081002&jour2=18&mois2=5&annee2=2020
['france', 'Douvaine_74', '74105002', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=74105002&jour2=18&mois2=5&annee2=2020
['france', 'Douzens_11', '218', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=218&jour2=18&mois2=5&annee2=2020
['france', 'Douzy_08', '111', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=111&jour2=18&mois2=5&annee2=2020
['france', 'Douzy_08', '8145001', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=8145001&jour2=18&mois2=5&annee2=2020
['france', 'Dozul

['france', 'Eze_06', '6059003', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=6059003&jour2=18&mois2=5&annee2=2020
['france', 'Fagnieres_51', '51242001', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=51242001&jour2=18&mois2=5&annee2=2020
['france', 'Fains-veel_55', '67', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=67&jour2=18&mois2=5&annee2=2020
['france', 'Fameck_57', '276', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=276&jour2=18&mois2=5&annee2=2020
['france', 'Fanjeaux_11', '11136001', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=11136001&jour2=18&mois2=5&annee2=2020
['france', 'Fargues-sur-ourbise_47', '47093002', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=47093002&jour2=18&mois2=5&annee2=2020
['f

['france', 'Gap_05', '5061009', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=5061009&jour2=18&mois2=5&annee2=2020
['france', 'Gap-sud_05', '74', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=74&jour2=18&mois2=5&annee2=2020
['france', 'Gatuzieres_48', '48069001', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=48069001&jour2=18&mois2=5&annee2=2020
['france', 'Gavarnie_65', '65188005', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=65188005&jour2=18&mois2=5&annee2=2020
['france', 'Geishouse_68', '68102001', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=68102001&jour2=18&mois2=5&annee2=2020
['france', 'Generargues_30', '30129001', 'Asia/Almaty']
Retrieving data from: http://www.meteociel.fr/temps-reel/obs_villes.php?code2=30129001&jour2=18&mois2=5&annee2=2020


KeyboardInterrupt: 